In [1]:
import numpy as np
import pandas
import os
from urllib.request import urlretrieve
import zipfile

In [2]:
import os
import csv
import gzip
import collections
import re
import io
import json
import xml.etree.ElementTree as ET
import untangle

import requests
import pandas as pd

In [6]:
zip_ref = zipfile.ZipFile(filename, 'r')
zip_ref.extractall()
zip_ref.close()

In [7]:
filename="full database.xml"
obj=untangle.parse(filename)

In [8]:
df_drugbank_sm=pd.DataFrame(columns=["drugbank_id","name","cas","smiles","logP ALOGPS", "logP ChemAxon", "solubility ALOGPS", "pKa (strongest acidic)", "pKa (strongest basic)"])
df_drugbank_sm

,drugbank_id,name,cas,smiles,logP ALOGPS,logP ChemAxon,solubility ALOGPS,pKa (strongest acidic),pKa (strongest basic)


In [9]:
i=-1
#iterate over drug entries to extract information
for drug in obj.drugbank.drug:
    drug_type= str(drug["type"])
    
    # select for small molecule drugs
    if drug_type in ["small molecule", "Small Molecule", "Small molecule"]:
        i=i+1    
        
        #Get drugbank_id
        for id in drug.drugbank_id:
            if str(id["primary"])=="true":
                df_drugbank_sm.loc[i, "drugbank_id"]=id.cdata
        #Drug name
        df_drugbank_sm.loc[i,"name"]=drug.name.cdata
        
        #Drug CAS
        df_drugbank_sm.loc[i, "cas"]=drug.cas_number.cdata
        
        #Get SMILES, logP, Solubility
        #Skip drugs with no structure. ("DB00386","DB00407","DB00702","DB00785","DB00840",
        #                                            "DB00893","DB00930","DB00965", "DB01109","DB01266",
        #                                           "DB01323", "DB01341"...)
        if len(drug.calculated_properties.cdata)==0: #If there is no calculated properties
            continue
        else:
            for property in drug.calculated_properties.property:
                if property.kind.cdata == "SMILES":
                    df_drugbank_sm.loc[i, "smiles"]=property.value.cdata
                    
                if property.kind.cdata == "logP":
                    if property.source.cdata == "ALOGPS":
                        df_drugbank_sm.loc[i, "logP ALOGPS"]=property.value.cdata
                    if property.source.cdata == "ChemAxon":
                        df_drugbank_sm.loc[i, "logP ChemAxon"]=property.value.cdata
                
                if property.kind.cdata == "Water Solubility":
                    df_drugbank_sm.loc[i, "solubility ALOGPS"]=property.value.cdata
                
                if property.kind.cdata == "pKa (strongest acidic)":
                    df_drugbank_sm.loc[i, "pKa (strongest acidic)"]=property.value.cdata
                
                if property.kind.cdata == "pKa (strongest basic)":
                    df_drugbank_sm.loc[i, "pKa (strongest basic)"]=property.value.cdata

In [10]:
df_drugbank_sm.head()

,drugbank_id,name,cas,smiles,logP ALOGPS,logP ChemAxon,solubility ALOGPS,pKa (strongest acidic),pKa (strongest basic)
0,DB00006,Bivalirudin,128270-60-0,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,-0.76,-14,4.64e-02 g/l,2.79,11.88
1,DB00014,Goserelin,65807-02-5,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,0.3,-5.2,2.83e-02 g/l,9.27,10.82
2,DB00027,Gramicidin D,1405-97-6,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,4.38,5.96,3.90e-03 g/l,11.56,NaN
3,DB00035,Desmopressin,16679-58-6,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,-1,-6.1,1.10e-01 g/l,9.5,11.77
4,DB00050,Cetrorelix,120287-85-6,CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H...,1.33,-1.7,6.94e-03 g/l,9.49,11.11


In [12]:
#Drop drugs without SMILES from the dataframe
df_drugbank_smiles = df_drugbank_sm.dropna()
df_drugbank_smiles= df_drugbank_smiles.reset_index(drop=True)
print(df_drugbank_smiles.shape)

(7551, 9)


In [14]:
df_drugbank_smiles.to_csv('drugbank_smiles_df.csv',index=False)